In [10]:
import os

import pandas as pd

import pingouin as pg

import numpy as np

from tqdm import tqdm

from concurrent.futures import ThreadPoolExecutor

NUM_WORKERS = 2

c:\users\sithi\research\myenv2\lib\site-packages\outdated\utils.py:18: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


# Utilities

In [ ]:
def get_columns(df, masks):
    
    out_columns = []

    for column in df.columns.to_list():

        for mask in masks:

            flag = False

            if mask in column:
                flag = True
                break;
        if not flag:
            out_columns.append(column)
            
    return out_columns

In [ ]:
def compute_icc(df, features):
    
    pbar = tqdm(range(len(features)*len(SOI)), desc=f"Computing ICC", position=0)

    icc_df = {}

    for sequence in SOI:

        _df = df[df.sequence==sequence][features+["id","judge"]]

        for feat in features:

            filter_name, feat_family_name, feat_name = feat.split("_")

            icc = pg.intraclass_corr(data=_df, targets='id',raters='judge', ratings=feat)

            icc_row = icc[icc.Type=="ICC1"]#ICC(1,1)

            icc_val = np.round(icc_row.ICC.item(),2)
            ci_down = np.round(icc_row["CI95%"].item()[0],2)
            ci_up = np.round(icc_row["CI95%"].item()[1],2)


            icc_df.setdefault("sequence",[]).append(sequence)
            icc_df.setdefault("filter",[]).append(filter_name)
            icc_df.setdefault("feat_family",[]).append(feat_family_name)
            icc_df.setdefault("feature",[]).append(feat_name)
            icc_df.setdefault("icc_value",[]).append(icc_val if icc_val>0 else 0)
            icc_df.setdefault("ci_down",[]).append(ci_down if ci_down>0 else 0)
            icc_df.setdefault("ci_up",[]).append(ci_up if ci_up>0 else 0)
            
            pbar.update()


    icc_df = pd.DataFrame.from_dict(icc_df)
    
    return icc_df

In [ ]:
def parse(aug_type):
    
    global ICC_DFS
    
    print(f"Analyzing stability for {aug_type}")
    
    df = pd.read_csv(os.path.join(FEAT_DIR,aug_type+"_"+FNAME_AFFIX))
    feats = get_columns(df, MASK_FEATS)
    
    ICC_DFS[aug_type] = compute_icc(df, feats)

# 1. Internal Population

In [ ]:
FEAT_DIR = r"C:\Users\sithi\Research\INT-Projects\stability\radiomicsFeatures"
OUT_DIR = r"C:\Users\sithi\Research\INT-Projects\stability\radiomicsFeaturesICC"

In [ ]:
MASK_FEATS = ["diagnostics","Unnamed",'id','judge','sequence', 'dice']
SOI = ["t2w","adc","sub_win","sub_wout"]
AUG_TYPES = ["in_plane_random","in_plane_systematic", "out_plane", "inout_plane_random","inout_plane_systematic"]
FNAME_AFFIX = "internal.csv"

In [ ]:
ICC_DFS = {}

with ThreadPoolExecutor(max_workers=NUM_WORKERS) as e:e.map(parse, AUG_TYPES)
    
for aug_type in AUG_TYPES:
    icc_df = ICC_DFS[aug_type]
    icc_df.to_csv(os.path.join(OUT_DIR,f"{aug_type}_{FNAME_AFFIX}"))

In [ ]:
# ICC_DFS = {}

# aug_type = "in_plane_random"

# parse(aug_type)

# icc_df = ICC_DFS[aug_type]
# icc_df.to_csv(os.path.join(OUT_DIR,f"{aug_type}_{FNAME_AFFIX}"))


# 2. External Population

In [3]:
FEAT_DIR = r"./radiomicsFeatures"
OUT_DIR = r"./radiomicsFeaturesICC"

In [4]:
MASK_FEATS = ["diagnostics","Unnamed",'id','judge','sequence', 'dice']
SOI = ["t2w","adc","sub"]
AUG_TYPES = ["in_plane_random","in_plane_systematic", "out_plane", "inout_plane_random","inout_plane_systematic"]
FNAME_AFFIX = "external.csv"

In [ ]:
ICC_DFS = {}

with ThreadPoolExecutor(max_workers=NUM_WORKERS) as e:e.map(parse, AUG_TYPES)
    
for aug_type in AUG_TYPES:
    icc_df = ICC_DFS[aug_type]
    icc_df.to_csv(os.path.join(OUT_DIR,f"{aug_type}_{FNAME_AFFIX}"))

# Heat Map Generation of UnFiltered Features & Filtered Features

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
heat_maps = {}

for sequence in SOI:
    
    heat_maps[sequence] = {}
    
    sequence_icc_df = icc_df[icc_df.sequence==sequence]
    
    _df = sequence_icc_df[["feat_family","filter","feature","ci_down"]]
    
    subplots = []
    

    
    cmap = sns.light_palette("seagreen", as_cmap=True)#sns.color_palette("Blues", as_cmap=True)#sns.light_palette("seagreen", as_cmap=True)#sns.color_palette("RdBu_r", as_cmap=True)
    #cmap = sns.cm.rocket_r#sns.light_palette("seagreen", as_cmap=True)
    #sns.heatmap(heatmap_arr, cmap=cmap,linewidths=2)
    
    for feat_family in _df.feat_family.unique():
   
        
        df1 = _df[_df.feat_family==feat_family]
        arr = df1.sort_values('feature').ci_down.to_numpy()
        stable_feats = arr>=0.9
   
        stable_feats[arr==np.nan] = np.nan
        
        m = int(np.ceil(np.sqrt(len(arr))))

        plt.figure(figsize=(m,m))
        square_arr = np.pad(arr, (0,(m*m)-len(arr)), mode='constant',constant_values=np.nan).reshape(m,m)
        
        heat_maps[sequence][feat_family] = {}
        
        stable_feats = (square_arr>=0.9)
        
        
        sns.heatmap(square_arr, vmin=0, vmax=1,linewidths=2, mask =~stable_feats,square=True, cmap=cmap)
        sns.heatmap(square_arr, vmin=0, vmax=0.9, linewidths=2, mask = stable_feats, square=True, cmap = sns.dark_palette("gray",as_cmap=True),cbar=False)
        
        #sns.heatmap(np.isnan(square_arr), vmin=0, vmax=1, linewidths=2, mask = ~np.isnan(square_arr), square=True, cmap=sns.dark_palette("gray",as_cmap=True))
        #sns.heatmap(np.isnan(square_arr), vmin=0, vmax=1, linewidths=2, mask = !np.isnan(square_arr), square=True, cmap=)

        plt.show()
        
    
    #filtered_df = sequence_icc_df[sequence_icc_df["filter"]!="original"]
    
    break;

In [ ]:
sns.bar

In [ ]:
Impact of HeatMap Strategies

In [ ]:
for sequence in SOI:
    
    sequence_icc_df = icc_df[icc_df.sequence==sequence]
    
    _df = sequence_icc_df[sequence_icc_df["filter"]=="original"][["feat_family","feature","ci_down"]]
    
    subplots = []
    
    #cmap = sns.color_palette("RdBu_r", as_cmap=True)
    #cmap = sns.cm.rocket_r#sns.light_palette("seagreen", as_cmap=True)
    #sns.heatmap(heatmap_arr, cmap=cmap,linewidths=2)
    
    for feat_family in _df.feat_family.unique():
        
        df1 = _df[_df.feat_family==feat_family]
        arr = df1.sort_values('feature').ci_down.to_numpy()
        
        m = int(np.ceil(np.sqrt(len(arr))))

        plt.figure(figsize=(m,m))
        square_arr = np.pad(arr, (0,(m*m)-len(arr)), mode='constant',constant_values=np.nan).reshape(m,m)
        sns.heatmap(square_arr, vmin=0, vmax=1, linewidths=2, mask = np.isnan(square_arr),square=True)
        plt.show()
        
    
    #filtered_df = sequence_icc_df[sequence_icc_df["filter"]!="original"]
    
    break;

In [ ]:
display(subplots[0])

In [ ]:
heatmap_arr[heatmap_arr==0] = np.nan

In [ ]:
#sns.color_palette("rocket")
cmap = sns.color_palette("RdBu_r", as_cmap=True)
sns.heatmap(heatmap_arr, cmap=cmap,linewidths=2, mask = np.isnan(heatmap_arr))


In [ ]:
4*4 for shape

5*5 for firstorder
5*5 for glcm
4x4 for glrlm
4x4 for glszm
3x3 for ngtdm
4x4 for gldm

In [ ]:
unfiltered_df

In [ ]:
5*5

In [ ]:
len(unfiltered_df)

In [ ]:
sns.heatmap(unfiltered_df)

In [ ]:
sns.clustermap(df_heatmap)

In [ ]:
net = Network(clustergrammer_widget)

In [ ]:
net.load_df(unfiltered_df)

In [ ]:
net.make_clust()

In [ ]:
net.widget()

In [ ]:
net = Network(CGM2)

In [ ]:
net.load_df(unfiltered_df)

In [ ]:
_df = pd.DataFrame(unfiltered_df.ci_down, index=unfiltered_df.feature, columns=unfiltered_df.feat_family)

In [ ]:
sns.heatmap(_df)